In [1]:
import qiskit as qk
import qutip as qt
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
from pylanczos import PyLanczos
import scipy.optimize as so
import scipy.sparse as scs
import math

In [2]:
def TFIM_multiply(psi, phi, N, J, Gamma):
    dim = 2 ** N
    for state in range(dim):
        jtotal = 0
        for site in range(N - 1):
            jtotal += J if ((state >> site) ^ (state >> (site + 1))) & 1 else -J 
        jtotal += J if ((state >> (N - 1)) ^ (state >> 0)) & 1 else -J
        phi[state] = jtotal * psi[state]
    
    for state in range(dim):
        for site in range(N):
            flipped_state = state ^ (1 << site)
            phi[flipped_state] -= Gamma*psi[state]

In [3]:
def TFIM_expectation(psi, N, J, Gamma):
    dim = 2 ** N
    phi = np.zeros(dim, dtype=np.complex128)
    psi = np.array(psi)
    bra_psi = psi.reshape((1, -1)).conj()
    TFIM_multiply(psi, phi, N, J, Gamma)
    return (bra_psi @ phi.reshape((-1, 1)))[0][0]

In [4]:
TFIM_expectation([0.5, 0.5, 0.5, 0.5], 2, 1, 1)

np.complex128(-2+0j)

In [6]:
qt.tensor([qt.basis(2, 0), qt.basis(2, 1)])

Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]]

In [7]:
def generate_state(state_num, N):
    x = np.zeros((N, 1))
    for n in range(N):
        x[n][0] = (state_num >> n) & 1
    return x

def sigmoid(x):
    return np.array([math.copysign(1, n[0]) * 1 if abs(n[0]) > 10 else 1/(1 + np.exp(-n[0])) for n in x]).reshape((-1, 1))

def compute_NN(weights, layers, N, J, Gamma):
    # weights as 1D array, first N * N numbers are first layer weights
    # next N numbers are first layer biases, next 2 * N numbers are second
    # layer weights, next 2 numbers are second layer biases.
    results = []
    dim = 2 ** N

    weights = np.array(weights)
    weight_arr = []
    bias_arr = []
    total = 0
    for i in range(1, len(layers)):
        weight_arr.append(weights[total:(total + layers[i] * layers[i - 1])].reshape((layers[i], layers[i - 1])))
        total += layers[i] * layers[i - 1]
        bias_arr.append(weights[total:(total + layers[i])].reshape((-1, 1)))
        total += layers[i]
    for state in range(dim):
        vector = generate_state(state, N)
        for i in range(len(weight_arr)):
            vector = sigmoid(weight_arr[i] @ vector + bias_arr[i])
        results.append(vector[0][0] + vector[1][0] * 1.j)
    
    mag = sum(abs(n) ** 2 for n in results)

    expect = TFIM_expectation(results, N, J, Gamma) / mag
    return expect.real 


In [8]:
def number_weights(layers):
    total = 0
    for i in range(1, len(layers)):
        total += layers[i] * layers[i - 1] + layers[i]
    return total

In [9]:
N = 7
J = 1
Gamma = 1
layers = [N, 2 * N, 2]
num_weights = number_weights(layers)
init_weights = np.ones(num_weights) * -1
# init_weights = -0.00001 * np.random.rand(num_weights)
compute_NN(init_weights, layers, N, J, Gamma)

np.float64(-6.671695339521821)

In [10]:
res = so.minimize(lambda w : compute_NN(w, layers, N, J, Gamma), init_weights, method = 'Powell')
# default BFGS is pretty bad, Nelder-Mead and COBYLA are worth considering
# accuracy, Powell >> Nelder-Mead > COBYLA

In [11]:
res

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -8.889796185698815
       x: [ 4.906e+00 -3.364e+00 ... -8.181e-01 -7.973e-01]
     nit: 29
   direc: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
           ...
           [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
           [ 5.878e-02 -1.938e-04 ... -1.510e-04  1.601e-03]]
    nfev: 92849

In [12]:
def TFIM_hamiltonian(n, J, gamma):
    # uses ring shape
    id = qt.qeye(2)
    z = qt.sigmaz()
    x = qt.sigmax()
    sxi = []
    szi = []
    for i in range(n):
        sxi.append(qt.tensor([id] * i + [x] + [id] * (n - i - 1)))
        szi.append(qt.tensor([id] * i + [z] + [id] * (n - i - 1)))
    return -J * sum(szi[i] * szi[i + 1] for i in range(n - 1)) - J * szi[n - 1] * szi[0] - gamma * sum(sxi[i] for i in range(n))

In [13]:
h = TFIM_hamiltonian(N, J, Gamma)
h.eigenstates()

(array([-8.98791841e+00, -8.76257254e+00, -7.20775094e+00, -7.02703758e+00,
        -7.02703758e+00, -5.63524661e+00, -5.63524661e+00, -5.60387547e+00,
        -5.60387547e+00, -5.60387547e+00, -5.60387547e+00, -5.29150262e+00,
        -4.86286089e+00, -4.86286089e+00, -4.49395921e+00, -4.49395921e+00,
        -4.49395921e+00, -4.49395921e+00, -4.09783468e+00, -4.09783468e+00,
        -4.00000000e+00, -3.89971165e+00, -3.89971165e+00, -3.89971165e+00,
        -3.89971165e+00, -3.12732593e+00, -3.12732593e+00, -3.12732593e+00,
        -3.12732593e+00, -2.89008374e+00, -2.89008374e+00, -2.89008374e+00,
        -2.89008374e+00, -2.50792068e+00, -2.49395921e+00, -2.49395921e+00,
        -2.21983253e+00, -2.16417669e+00, -2.16417669e+00, -1.78016747e+00,
        -1.73553496e+00, -1.73553496e+00, -1.73553496e+00, -1.73553496e+00,
        -1.39179097e+00, -1.39179097e+00, -1.38404294e+00, -1.38404294e+00,
        -1.10991626e+00, -1.10991626e+00, -1.10991626e+00, -1.10991626e+00,
        -9.6

In [14]:
weights = res.x

results = []
dim = 2 ** N

weights = np.array(weights)
weight_arr = []
bias_arr = []
total = 0
for i in range(1, len(layers)):
    weight_arr.append(weights[total:(total + layers[i] * layers[i - 1])].reshape((layers[i], layers[i - 1])))
    total += layers[i] * layers[i - 1]
    bias_arr.append(weights[total:(total + layers[i])].reshape((-1, 1)))
    total += layers[i]
for state in range(dim):
    vector = generate_state(state, N)
    for i in range(len(weight_arr)):
        vector = sigmoid(weight_arr[i] @ vector + bias_arr[i])
    results.append(vector[0][0] + vector[1][0] * 1.j)

mag = sum(abs(n) ** 2 for n in results)

In [15]:
gs = np.array(results) / np.sqrt(mag)
gs

array([0.01144189+0.0116591j , 0.00889339+0.00896819j,
       0.01334866+0.01354198j, 0.01695763+0.01708747j,
       0.01646517+0.01672745j, 0.01202436+0.01207349j,
       0.01905666+0.01899062j, 0.02654491+0.02618734j,
       0.0102627 +0.01010849j, 0.0049799 +0.00487262j,
       0.00763743+0.0075098j , 0.01223539+0.0119991j ,
       0.0178949 +0.0176884j , 0.01347814+0.01326002j,
       0.03178651+0.03138742j, 0.04954352+0.04880376j,
       0.01640556+0.01663681j, 0.0092846 +0.00926387j,
       0.01064101+0.01057068j, 0.01093401+0.01077121j,
       0.00994488+0.00984172j, 0.00627265+0.00616036j,
       0.01534652+0.01514499j, 0.02303348+0.02266558j,
       0.02491415+0.02464029j, 0.01346498+0.01323382j,
       0.01815581+0.01794136j, 0.02361684+0.02327269j,
       0.03380371+0.03345668j, 0.02425262+0.02388069j,
       0.05015332+0.04961396j, 0.07231743+0.07142979j,
       0.01567122+0.01587423j, 0.00983462+0.00990888j,
       0.01205971+0.01212165j, 0.01657068+0.01651483j,
       0.0

In [16]:
qt.fidelity(qt.Qobj([abs(s) for s in gs]), h.eigenstates()[1][0])

np.float64(0.7797638654051632)

In [17]:
qt.Qobj([abs(s) for s in gs])

Quantum object: dims=[[128], [1]], shape=(128, 1), type='ket', dtype=Dense
Qobj data =
[[0.01633559]
 [0.01263016]
 [0.01901505]
 [0.02407369]
 [0.02347146]
 [0.01703979]
 [0.02690353]
 [0.03728819]
 [0.01440502]
 [0.0069672 ]
 [0.01071109]
 [0.01713719]
 [0.02516162]
 [0.01890737]
 [0.0446716 ]
 [0.069544  ]
 [0.02336505]
 [0.01311576]
 [0.01499901]
 [0.01534834]
 [0.01399143]
 [0.00879183]
 [0.02156123]
 [0.03231516]
 [0.03504082]
 [0.01887961]
 [0.025525  ]
 [0.0331568 ]
 [0.04756091]
 [0.0340364 ]
 [0.07054715]
 [0.10164657]
 [0.02230646]
 [0.01396086]
 [0.01709886]
 [0.02339503]
 [0.01819305]
 [0.00937647]
 [0.01924157]
 [0.03443419]
 [0.01402554]
 [0.00747979]
 [0.01199044]
 [0.01732451]
 [0.01945043]
 [0.01683494]
 [0.03306077]
 [0.05875908]
 [0.03279369]
 [0.02145402]
 [0.01802452]
 [0.03304889]
 [0.01963633]
 [0.01582336]
 [0.03312479]
 [0.05249637]
 [0.05009618]
 [0.0336855 ]
 [0.03117054]
 [0.05170582]
 [0.07027095]
 [0.05822828]
 [0.10500669]
 [0.19443502]
 [0.0217962 ]
 [0

In [18]:
h.eigenstates()[1][0]

Quantum object: dims=[[2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1]], shape=(128, 1), type='ket', dtype=Dense
Qobj data =
[[0.48560198]
 [0.1379053 ]
 [0.1379053 ]
 [0.08462187]
 [0.1379053 ]
 [0.0441179 ]
 [0.08462187]
 [0.06937171]
 [0.1379053 ]
 [0.04134209]
 [0.0441179 ]
 [0.03164354]
 [0.08462187]
 [0.03164354]
 [0.06937171]
 [0.06937171]
 [0.1379053 ]
 [0.04134209]
 [0.04134209]
 [0.02886774]
 [0.0441179 ]
 [0.01608828]
 [0.03164354]
 [0.03164354]
 [0.08462187]
 [0.02886774]
 [0.03164354]
 [0.02886774]
 [0.06937171]
 [0.03164354]
 [0.06937171]
 [0.08462187]
 [0.1379053 ]
 [0.0441179 ]
 [0.04134209]
 [0.03164354]
 [0.04134209]
 [0.01608828]
 [0.02886774]
 [0.03164354]
 [0.0441179 ]
 [0.01608828]
 [0.01608828]
 [0.01608828]
 [0.03164354]
 [0.01608828]
 [0.03164354]
 [0.0441179 ]
 [0.08462187]
 [0.03164354]
 [0.02886774]
 [0.02886774]
 [0.03164354]
 [0.01608828]
 [0.02886774]
 [0.04134209]
 [0.06937171]
 [0.03164354]
 [0.03164354]
 [0.04134209]
 [0.06937171]
 [0.0441179 ]
 [0.0846218

In [19]:
h @ h.eigenstates()[1][0]

Quantum object: dims=[[2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1]], shape=(128, 1), type='ket', dtype=Dense
Qobj data =
[[-4.36455095]
 [-1.2394816 ]
 [-1.2394816 ]
 [-0.76057446]
 [-1.2394816 ]
 [-0.39652806]
 [-0.76057446]
 [-0.62350728]
 [-1.2394816 ]
 [-0.37157935]
 [-0.39652806]
 [-0.28440959]
 [-0.76057446]
 [-0.28440959]
 [-0.62350728]
 [-0.62350728]
 [-1.2394816 ]
 [-0.37157935]
 [-0.37157935]
 [-0.25946088]
 [-0.39652806]
 [-0.14460014]
 [-0.28440959]
 [-0.28440959]
 [-0.76057446]
 [-0.25946088]
 [-0.28440959]
 [-0.25946088]
 [-0.62350728]
 [-0.28440959]
 [-0.62350728]
 [-0.76057446]
 [-1.2394816 ]
 [-0.39652806]
 [-0.37157935]
 [-0.28440959]
 [-0.37157935]
 [-0.14460014]
 [-0.25946088]
 [-0.28440959]
 [-0.39652806]
 [-0.14460014]
 [-0.14460014]
 [-0.14460014]
 [-0.28440959]
 [-0.14460014]
 [-0.28440959]
 [-0.39652806]
 [-0.76057446]
 [-0.28440959]
 [-0.25946088]
 [-0.25946088]
 [-0.28440959]
 [-0.14460014]
 [-0.25946088]
 [-0.37157935]
 [-0.62350728]
 [-0.28440959]
 [-0.2844

In [20]:
h.full() @ qt.Qobj([abs(s) for s in gs]).full()

array([[-0.25133851+0.j],
       [-0.16179686+0.j],
       [-0.18372518+0.j],
       [-0.24912504+0.j],
       [-0.20129535+0.j],
       [-0.1148252 +0.j],
       [-0.26302269+0.j],
       [-0.38670918+0.j],
       [-0.16438553+0.j],
       [-0.09794824+0.j],
       [-0.13225069+0.j],
       [-0.18130843+0.j],
       [-0.26123491+0.j],
       [-0.18304388+0.j],
       [-0.4028237 +0.j],
       [-0.58879831+0.j],
       [-0.21338544+0.j],
       [-0.10821249+0.j],
       [-0.11826299+0.j],
       [-0.16679927+0.j],
       [-0.13919314+0.j],
       [-0.09921054+0.j],
       [-0.18565914+0.j],
       [-0.25853932+0.j],
       [-0.31279033+0.j],
       [-0.16684203+0.j],
       [-0.18785056+0.j],
       [-0.24299742+0.j],
       [-0.42732175+0.j],
       [-0.27428759+0.j],
       [-0.61082675+0.j],
       [-0.93272065+0.j],
       [-0.21514359+0.j],
       [-0.12784051+0.j],
       [-0.11771629+0.j],
       [-0.18232204+0.j],
       [-0.11796061+0.j],
       [-0.10044104+0.j],
       [-0.1

In [21]:
h.full()

array([[-7.+0.j, -1.+0.j, -1.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
       [-1.+0.j, -3.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
       [-1.+0.j,  0.+0.j, -3.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
       ...,
       [ 0.+0.j,  0.+0.j,  0.+0.j, ..., -3.+0.j,  0.+0.j, -1.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j, -3.+0.j, -1.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, ..., -1.+0.j, -1.+0.j, -7.+0.j]])